In [1]:
from keras.models import load_model
import h5py
import numpy as np
import cv2
import pandas as pd
import pickle

Using TensorFlow backend.


Images are too big to be fed into a neural network. Resize them using ImageMagick Library to 64 by 36 pixels. 

Remember to drop closing credit scenes

In [3]:
#load autoencoder
autoencoder = load_model('video_autoencoder')
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 36, 64, 32)    2624      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 18, 32, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 18, 32, 64)    55360     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 7, 9, 16, 64)      0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 7, 9, 16, 64)      110656    
_________________________________________________________________
up_sampling3d_1 (UpSampling3 (None, 14, 18, 32, 64)    0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 14, 18, 32, 32)    55328     
__________

In [4]:
#import movie list
movieRuntimesPath = 'Numerical Data//movie_runtimes.csv'
movieRuntimeDf = pd.read_csv(movieRuntimesPath, usecols = ['movie', 'runtime (mins)', 'effective runtime'], nrows = 9)
movieList = list(movieRuntimeDf['movie'])

In [5]:
movieRuntimeDf

,movie,runtime (mins),effective runtime
0,Hobbit 2,160,302
1,Buddy,90,170
2,Machete Kills,108,204
3,Walter Mitty,114,209
4,Paranormal Activity,101,191
5,The Hunger Games-Catching Fire,146,272
6,Star Wars-The Force Awakens,136,254
7,"Help, I Shrunk My Teacher",97,181
8,I'm Off Then,89,168


In [7]:
width = 64
height = 36

movieFrameNumbers = dict()
movieFrameNumbers['Hobbit 2'] = 9678
movieFrameNumbers['Buddy'] = 5450
movieFrameNumbers['Machete Kills'] = 6481
movieFrameNumbers['Walter Mitty'] = 6875
movieFrameNumbers['Paranormal Activity'] = 6065
movieFrameNumbers['The Hunger Games-Catching Fire'] = 8775
movieFrameNumbers['Star Wars-The Force Awakens'] = 8287
movieFrameNumbers["Help, I Shrunk My Teacher"] = 5825
movieFrameNumbers["I'm Off Then"] = 5331

numOfFramesPerSeq = 30
#number of sequences is equal to the effective runtime

movieSequenceDict = dict()

#load images
#create a 5 dimension np array per movie - (num of sequences, 30 frames per section, 36, 64,3)
for movie in movieList:
    inputPath  = '/home/sof/Notebooks/disk/Features/Autoencoder Movie Frames/' + movie + '/'
    movieSeq = np.array([])
    for j in range(0, (movieFrameNumbers[movie])//numOfFramesPerSeq):
        imgSequence = np.array([])
        for imgIndex in range(((j*numOfFramesPerSeq)+1), ((j+1)*numOfFramesPerSeq)+1):
            #import image
            number = '{:04d}'.format(imgIndex)
            photoPath = inputPath + movie + "_" + str(number) + '.jpg'
            img = cv2.imread(photoPath)
            img = np.array(img, dtype=np.uint8)
            img = cv2.cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            #concatenate images onto the 4th dimension
            if imgIndex == ((j*numOfFramesPerSeq)+1):
                #starting new sequence
                imgSequence = img
                imgSequence = np.expand_dims(imgSequence, axis=0)
            else:
                #add to the same sequence
                img = np.expand_dims(img, axis=0)
                imgSequence = np.append(imgSequence, img, axis=0)
        #assign sequence to array
        if len(movieSeq) == 0: #empty movie sequences
            movieSeq = imgSequence
            movieSeq = np.expand_dims(movieSeq, axis=0)
        else:
            imgSequence = np.expand_dims(imgSequence, axis=0)
            movieSeq = np.append(movieSeq, imgSequence, axis=0)
           
    movieSequenceDict[movie] = movieSeq

In [9]:
condensedFeaturesDict = dict()
for movie in movieList:
    movieSequence = movieSequenceDict[movie]
    condensedFeatures = autoencoder.predict(movieSequence)
    condensedFeaturesDict[movie] = condensedFeatures
    #save as pickle objects
    savePath = movie + '.p'
    pickle.dump(condensedFeatures, open(savePath, "wb"))